<a href="https://colab.research.google.com/github/mmadduri/BMI_Model/blob/brainAdaptOnly/Winter2020_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Start with this:**

Run the (***simple random search***) iteration

$$ u^+ = u - \frac{\gamma}{N} \sum_{n = 1}^N ( c(u + u_n) - c(u) ) \cdot u_n,\ u_n \sim \mathcal{N}(u,\sigma^2) $$

In our case, the cost function is just the reach error -- calculated by the reach error between the previous lamdba values and the current one. 


Error is defined as reach error: $ error = ||t-y||^2 $ and the perturbation term $ p_{2k+1} $ can be thought of a normal distribution.

$$ \lambda^+ = \lambda - \frac{\nu}{N\delta} \sum_{n = 1}^N ( error(\lambda + \delta p) - error(\lambda) ) \cdot  p, p \sim \mathcal{N}(\lambda,\sigma^2) $$

In [0]:
import numpy as np
import matplotlib.pyplot as plt


# Function that generates the adaptation



####################
# Brain Model 
def calcNextLambda(lambda_vect, gamma, delta_perturb, targ_vect):
  # This is the vector to multiply the lambda update term with = [1 t_x t_y]'
  targ_vect_mult = np.insert(targ_vect, 0, 1)
  return lambda_vect - (gamma*delta_perturb*targ_vect_mult)

def brainFiringRate(lambda_vect, targ_vect):
  # f = b + Fwt 
  # [b w_x w_y]*[1 t_x  t_y ]'
  targ_vect_mult = np.insert(targ_vect, 0, 1)
  newFR = np.matmul(lambda_vect, targ_vect_mult) 
  return newFR

####################
# Decoder Model 
# K_matx = num neurons x num target 
K_matx = np.array([[0.5, 0.2]])

# keyword arguments--like a static variable (add states to function)
# make it's optional argument (kwargs)
# might have to make neurons_num global
# firing rate --> a + Kf --> y
def decoder_findY(firing_rate, targ_vect):
  global K_matx
  # D(f) = a + Kf = y
  # Start with affine decoder
  a_vect = np.ones(np.size(targ_vect))*0.5

  nextPred_vect = a_vect + np.dot(K_matx, np.squeeze(np.asarray(firing_rate)))
  return (nextPred_vect)

# Note to self: this function is returning a + Kf = y accurately
# ######
# define reach error
# y_x, y_y = matrix of predicted cursor position
# t_x, t_y = matrix of target position
def calcReachError(y_vect, t_vect):
  norm_vect = np.array(y_vect) - np.array(t_vect)
  return (np.linalg.norm(norm_vect, 2)**2)

def reachError_FR(firing_rate, targ_vect):
  y_vect = decoder_findY(firing_rate, targ_vect)
  t_vect = targ_vect
  return calcReachError(y_vect, t_vect)  

# find next g
# define random search

# this is only 1 iteration of SGD
# takes in a vector lambda 
# start with scalar case
def findErrorGrad(fr_input, sigma, delta, targ_vect):
  N_sum = 0
  N_neurons = np.size(fr_input)
  N_samp = 100

  perturb_rand = np.random.normal(fr_input, sigma, N_samp)

  for iN in range(N_samp):
    N_sum = N_sum + (reachError_FR(fr_input + delta*perturb_rand[iN], targ_vect) - reachError_FR(fr_input, targ_vect))*perturb_rand[iN]

  errorGrad = N_sum/(N_samp*delta)
  # print(avgUpdate)
  return errorGrad

def findErrorGrad_lambda(lambda_input, sigma, delta, targ_vect):
  N_sum = 0
  N_samp = 1

  print("lambda_input" + str(lambda_input))

  fr_curr = brainFiringRate(lambda_input, targ_vect)

  perturb_rand = np.zeros((np.size(lambda_input), N_samp))

  for iL in range(np.size(lambda_input)):
    perturb_rand[iL, :] = np.random.normal(lambda_input[iL], sigma, N_samp) 
    for iN in range(N_samp):
      # generate new firing rate
      lambda_perturb = np.add(lambda_input, delta*perturb_rand[:, iN])
      print("lambda perturb = " + str(lambda_perturb))
      print("lambda_input" + str(lambda_input))
      print("perturb_rand" + str(perturb_rand))
      fr_perturb = brainFiringRate(lambda_perturb, targ_vect)
      N_sum = N_sum + (reachError_FR(fr_perturb, targ_vect) - reachError_FR(fr_curr, targ_vect))*perturb_rand[iN]

    avgUpdate = N_sum/(N_samp*delta)
  # print(avgUpdate)
  return avgUpdate

# param vect = what to update 
def gradDesc(fr_init, lambda_init, targ_vect, learn_rate, numIter, sigma, delta):
  fr_vect = np.array(fr_init)
  lambda_vect = np.zeros((np.size(lambda_init), numIter))
  lambda_vect[:,0] = lambda_init

  err_next = reachError_FR(fr_vect[-1], targ_vect)
  err_vect = np.array(err_next)

  delta_vect = np.array(err_next)
  
  for iT in range(1, numIter, 1):
    # delta_perturb = \Delta_feedback term due to perturbation
    # delta_perturb = findErrorGrad(fr_vect[-1], sigma, delta, targ_vect)

    # fr_fb_next = fr_vect[-1] + learn_rate*delta_perturb
    delta_perturb = findErrorGrad_lambda(lambda_vect[:, iT-1], sigma, delta, targ_vect)
    print("delta perturb = " + str(delta_perturb))

    
    # lambda+ = lambda - learn_rate*delta_perturb
    lambda_next = calcNextLambda(lambda_vect[:, iT-1], learn_rate, delta_perturb, targ_vect)
    print("next lambda = " + str(lambda_next))

    # # calculate new firing rate with lamdba+
    fr_next = np.abs(brainFiringRate(lambda_next, targ_vect))

    # calculate new reach error with lamdba+ and fr+
    err_next = np.array(reachError_FR(fr_next, targ_vect))  

    # update vectors 
    fr_vect = np.vstack( (fr_vect, fr_next) )
    err_vect = np.vstack( (err_vect, err_next) )  
    delta_vect = np.vstack( (delta_vect, delta_perturb) )
    # lambda_vect = np.vstack( (lambda_vect, lambda_next) )
    lambda_vect[:, iT] = lambda_next


  it_idx = np.linspace(0, numIter, numIter, endpoint=False) 
  plt.figure()
  fig, ax = plt.subplots()

  ax.plot(it_idx, err_vect, label = 'RE')
  ax.plot(it_idx, fr_vect, label = 'FR') 
  # ax.plot(it_idx, lambda_vect, label = 'lambda')
  leg = ax.legend();
  plt.grid()
  plt.title('stoch update')

  print("Final Reach Error = " + str(err_vect[numIter-1]) )


#### Code

##### TESTING ONLY
# define c, D1_c, D2_c
def cost(u):
  return ((u**6)/6) - (7/5 * u**5) + (17/4 * u**4) - (17/3 * u**3) + (3 * u**2)
def D1_c(u):
  return ((u**5) - (7 * u**4) + (17 * u**3) - (17 * u**2) + (6 * u**1))
def D2_c(u):
  return ((5 * u**4) - (7*4 * u** 3) + (17*3 * u**2) - (17*2 * u) + (6))

#define gradient descent 
def gradDesc_cont(rate, it_num, u_init):
  u_new = [u_init]
  c_new = [cost(u_new[-1])]
  for it in range(1,it_num,1):
    u_new.append(u_new[-1] - rate*D1_c(u_new[-1]))
    c_new.append(cost(u_new[-1]))
  it_idx = np.linspace(0,it_num, it_num, endpoint=False) 
  plt.figure()
  plt.plot(it_idx, c_new)
  plt.plot(it_idx, u_new)

# # Set initial conditions

# #######
# Set initial conditions
learn_rate = 0.5
numIter = 4
fr_init = [0.5]
lambda_init = np.array([0.1, 0.4, 3])
sigma = 1e-1
delta = 1e-3

TARGET_NUM = 2
TARGET_VECTOR = [2, 2]
gradDesc(fr_init, lambda_init, TARGET_VECTOR, learn_rate, numIter, sigma, delta)

# rate = 0.1
# u_init = 0.5
# gradDesc_cont(rate, numIter, u_init)
# plt.legend({'c', 'u'})
# plt.grid()

# e1 = decoder_findY(u_init, TARGET_VECTOR)
# print(e1)
# print(reachError_FR(u_init, TARGET_VECTOR))
# print(np.linalg.norm(e1, 2)**2)

In [59]:
import numpy as np
import matplotlib.pyplot as plt


# Function that generates the adaptation



####################
# Brain Model 
def calcNextLambda(lambda_vect, gamma, delta_perturb, targ_vect):
  # This is the vector to multiply the lambda update term with = [1 t_x t_y]'
  targ_vect_mult = np.insert(targ_vect, 0, 1)
  return lambda_vect - (gamma*delta_perturb*targ_vect_mult)

## lambda, t --> f
def brainFiringRate(lambda_vect, targ_vect):
  # f = b + Fwt 
  # [b w_x w_y]*[1 t_x  t_y ]'
  targ_vect_mult = np.insert(targ_vect.copy(), 0, 1)
  newFR = np.matmul(lambda_vect, targ_vect_mult) 
  return newFR

####################
# Decoder Model 
# K_matx = num neurons x num target 
K_matx = np.array([[0.5, 0.2]])

# keyword arguments--like a static variable (add states to function)
# make it's optional argument (kwargs)
# might have to make neurons_num global
# firing rate --> a + Kf --> y
## f --> y
def decoder_findY(firing_rate, targ_vect):
  global K_matx
  # D(f) = a + Kf = y
  # Start with affine decoder
  a_vect = np.ones(np.size(targ_vect))*0.5

  nextPred_vect = a_vect + np.dot(K_matx, np.squeeze(np.asarray(firing_rate)))
  return (nextPred_vect)

# Note to self: this function is returning a + Kf = y accurately
# ######
# define reach error
# y_x, y_y = matrix of predicted cursor position
# t_x, t_y = matrix of target position
def calcReachError(y_vect, t_vect):
  norm_vect = np.array(y_vect) - np.array(t_vect)
  return (np.linalg.norm(norm_vect, 2)**2)

def reachError_FR(firing_rate, targ_vect):
  y_vect = decoder_findY(firing_rate, targ_vect)
  t_vect = targ_vect
  return calcReachError(y_vect, t_vect)  

# find next g
# define random search

# this is only 1 iteration of SGD
# takes in a vector lambda 
# start with scalar case
def findErrorGrad(fr_input, sigma, delta, targ_vect):
  N_sum = 0
  N_neurons = np.size(fr_input)
  N_samp = 100

  perturb_rand = np.random.normal(fr_input, sigma, N_samp)

  for iN in range(N_samp):
    N_sum = N_sum + (reachError_FR(fr_input + delta*perturb_rand[iN], targ_vect) - reachError_FR(fr_input, targ_vect))*perturb_rand[iN]

  errorGrad = N_sum/(N_samp*delta)
  # print(avgUpdate)
  return errorGrad

def findErrorGrad_lambda(lambda_input, sigma, delta, targ_vect):
  N_samp = 100
  lambda_size = np.size(lambda_input)
  errorGrad = np.zeros(lambda_size)
  lambda_perturb = np.zeros(lambda_size)


  # fr_curr = brainFiringRate(lambda_input, targ_vect)

  perturb_rand = np.zeros(( lambda_size, N_samp ))
  for iL in range(lambda_size): 
    perturb_rand[iL, :] = np.random.normal(lambda_input[iL], sigma, N_samp) 
    N_sum = 0
    for iN in range(N_samp):
      # lambda_curr = lambda_input[:]
      lambda_perturb = lambda_input.copy()
      # perturb_term = lambda_perturb[iL] + delta*perturb_rand[iL, iN]
      # lambda_perturb[iL] = lambda_perturb[iL] + delta*perturb_rand[iL, iN]
      lambda_perturb[iL] = delta*perturb_rand[iL, iN]

    

      error_perturb = calcReachError(decoder_findY(brainFiringRate(lambda_perturb, targ_vect), targ_vect), targ_vect)
      error_input = calcReachError(decoder_findY(brainFiringRate(lambda_input, targ_vect), targ_vect), targ_vect)

      # print everything
      # print( "lambda_curr = " + str(lambda_input) )
      # print( "lambda_perturb = " + str(lambda_perturb) )
      # print("b_FR perturb = " + str(brainFiringRate(lambda_perturb, targ_vect) ))
      # print("find Y perturb = " + str(decoder_findY(brainFiringRate(lambda_perturb, targ_vect), targ_vect)) )
      # print("find error perturb = " + str(error_perturb) )

      # print("b_FR input = " + str(brainFiringRate(lambda_input, targ_vect) ))
      # print("find Y input = " + str(decoder_findY(brainFiringRate(lambda_input, targ_vect), targ_vect)))
      # print("find error input = " + str(error_input))

      # print("error grad = " + str( (error_perturb - error_input)*perturb_rand[iL, iN]))
      N_sum = N_sum + (np.abs( calcReachError(decoder_findY(brainFiringRate(lambda_perturb, targ_vect), targ_vect), targ_vect ) 
                              - calcReachError(decoder_findY(brainFiringRate(lambda_input, targ_vect), targ_vect), targ_vect)))*perturb_rand[iL, iN]  
      # print("running sum = " + str(N_sum))
    errorGrad[iL] = N_sum/(N_samp*delta)
  
  # print(np.shape(errorGrad))
  return errorGrad

# param vect = what to update 
def gradDesc(fr_init, lambda_init, targ_vect, learn_rate, numIter, sigma, delta):
  fr_vect = np.array(fr_init)
  lambda_vect = np.zeros((np.size(lambda_init), numIter))
  lambda_vect[:,0] = lambda_init

  err_next = reachError_FR(fr_vect[-1], targ_vect)
  err_vect = np.array(err_next)

  delta_vect = np.array(err_next)
  
  for iT in range(1, numIter, 1):
    # delta_perturb = \Delta_feedback term due to perturbation
    # delta_perturb = findErrorGrad(fr_vect[-1], sigma, delta, targ_vect)

    # fr_fb_next = fr_vect[-1] + learn_rate*delta_perturb
    delta_perturb = findErrorGrad_lambda(lambda_vect[:, iT-1], sigma, delta, targ_vect)
    print("delta perturb = " + str(delta_perturb))

    
    # lambda+ = lambda - learn_rate*delta_perturb
    lambda_next = calcNextLambda(lambda_vect[:, iT-1], learn_rate, delta_perturb, targ_vect)
    print("next lambda = " + str(lambda_next))

    # # calculate new firing rate with lamdba+
    fr_next = np.abs(brainFiringRate(lambda_next, targ_vect))

    # calculate new reach error with lamdba+ and fr+
    err_next = np.array(reachError_FR(fr_next, targ_vect))  

    # update vectors 
    fr_vect = np.vstack( (fr_vect, fr_next) )
    err_vect = np.vstack( (err_vect, err_next) )  
    delta_vect = np.vstack( (delta_vect, delta_perturb) )
    # lambda_vect = np.vstack( (lambda_vect, lambda_next) )
    lambda_vect[:, iT] = lambda_next


  it_idx = np.linspace(0, numIter, numIter, endpoint=False) 
  plt.figure()
  fig, ax = plt.subplots()

  ax.plot(it_idx, err_vect, label = 'RE')
  ax.plot(it_idx, fr_vect, label = 'FR') 
  # ax.plot(it_idx, lambda_vect, label = 'lambda')
  leg = ax.legend();
  plt.grid()
  plt.title('stoch update')

  print("Final Reach Error = " + str(err_vect[numIter-1]) )


#### Code

##### TESTING ONLY
# define c, D1_c, D2_c
def cost(u):
  return ((u**6)/6) - (7/5 * u**5) + (17/4 * u**4) - (17/3 * u**3) + (3 * u**2)
def D1_c(u):
  return ((u**5) - (7 * u**4) + (17 * u**3) - (17 * u**2) + (6 * u**1))
def D2_c(u):
  return ((5 * u**4) - (7*4 * u** 3) + (17*3 * u**2) - (17*2 * u) + (6))

#define gradient descent 
def gradDesc_cont(rate, it_num, u_init):
  u_new = [u_init]
  c_new = [cost(u_new[-1])]
  for it in range(1,it_num,1):
    u_new.append(u_new[-1] - rate*D1_c(u_new[-1]))
    c_new.append(cost(u_new[-1]))
  it_idx = np.linspace(0,it_num, it_num, endpoint=False) 
  plt.figure()
  plt.plot(it_idx, c_new)
  plt.plot(it_idx, u_new)

# # Set initial conditions

# #######
# Set initial conditions
learn_rate = 0.5
numIter = 4
fr_init = [0.5]
lambda_init = np.array([0.1, 0.4, 3])
sigma = 1e-1
delta = 1

TARGET_NUM = 2
TARGET_VECTOR = [2, 2]
# gradDesc(fr_init, lambda_init, TARGET_VECTOR, learn_rate, numIter, sigma, delta)


##

print(findErrorGrad_lambda(lambda_init, sigma, delta, TARGET_VECTOR))

##### TESTING ONLY
# rate = 0.1
# u_init = 0.5
# gradDesc_cont(rate, numIter, u_init)
# plt.legend({'c', 'u'})
# plt.grid()

# e1 = decoder_findY(u_init, TARGET_VECTOR)
# print(e1)
# print(reachError_FR(u_init, TARGET_VECTOR))
# print(np.linalg.norm(e1, 2)**2)

[0.01655788 0.13316619 0.85032117]
